### Tree Recursive Neural Networks
目的是使能够识别意义相近的短语<br>
semantic compositionality: artificial Intelligence requires being able to understand bigger things from knowing about smaller parts. 这正是我们tree RNN的主要作用。<br>
这个方法的基础是：人类语言具有recursive的性质，如：S = PW|WP; P=W(S: Sentence P: Phrase W: word)当然一般语言不会这么简单<br>
主要思想是：和word vector类似，用一个vector来代表一个短语，使得，意思相近的短语，其对应的vectors具有相似性。一般这里短语的向量空间和原来word的向量空间相同，从而使其可以更好的模拟单词和短语之间的相识性。比如：a person on a snowboard, and snowboarder<br>

#### How should we map phrases into a vector space?
Use principle of ocmpositionality: 
The meaning(vector) of a sentence is determined by 
- the meaning of its words and 
- the rules that combine them
应此，我们希望有这么一个模型，他可以在为一个句子的构建句子结构的同时通过一种表达方式，表达这个句子的意义。(也就是说，在构建句法结构的同时构建语义结构。)

Recursive vs. recurrent neural networks
- The problem of Recursive is you require a parser to get tree structure. putting a tree stucture over a sentence is making deterministic categorcal choices as to which words are going together to be constituents while other words aren't. And anywhere you're making categorical choices that's a problem for leaning a model simply by running back propagation. And so that sort of put complexity inot these models. It also means that they're kind of GPU unfriendly, cause there isn't just sort of simple lock step computation, like an LSTM gives you.
- Recurrent neural nets cannot capture phrases without prefix context and often capture too much of last words in final vector. They have the disadvantage that they're not actually representingany of the structure of the sentence.

Relationship between RNNs and CNNs
- RNN: get compositional vectors for grammatical phrases only
- CNN: computes vectors for every possible phrase.(通过pooling选择最具代表性的) But it's not very linguistically or congnitively plausible

#### Recursive Neural Networks for structure prediction
Inputs: two candidate children's representations
Ouptuts:
- The semantic representation if the two nodes are merged
- Score of how plausible the new node would be.<br>
一种简单的实现方法如下：

![tree rnn](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/tree_rnn.png)

### Parse a sentence with an RNN
#### Max-margin Framework
以上面的方法为基础，我们可以用greedy的方法parse a sentence。每次结合得分最高的项，然后放回，直到只有一个项。
The score of a tree is computed by the sum of the parsing decision scores at each node:
<center>$s(x,y)=\sum_{n\in nodes(y)}s_n$</center>
x is sentence; y is parse tree<br>
Similar to max-margin parsing, a supervised max-margin objective:
<center>$J=\sum_i s(x_i,y_i) - max_{y\in A(x_i)}(s(x_i, y)+\Delta(y,y_i))$</center>
The loss $\Delta(y,y_i)$ penalize all incorrect decisions<br>
Structure search for A(x) was greedy(join best nodes each time)(Instead: Beam search with chart)
#### Backpropagation through Structure
Principally the same as general backpropagation:
<center>$\delta^{(l)}=((W^{(l)})^T\delta^{(l+1)})\ f'(z^{(l)})$</center>
<center>$\frac{\delta}{\delta W^{(l)}}E_R=\delta^{(l+1)}(a^{(l)})^T+\lambda W^{(l)}$</center>
Threee differences resulting from the recursion and tree structure:
- Sum derivatives of W from all nodes(like RNN)
- Split derivatives at each node(for tree)
- Add error messages from parent + node itself<br>
每个节点都有自己的error，当error从树根传递到当前节点的时候，当前节点的error为传递下来的节点加上节点自己的error。然后这个error会被分裂，传递到下一层的节点上。
贴公式，但实际上还没有看

![bpts1](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/backprop_through_structure1.png)
![bpts2](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/backprop_through_structure2.png)
![bpts3](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/backprop_through_structure3.png)

### highlight
这期的highlight是deep reinforcement Learning for Dialogue Generation(Jiwei Li, Will Monroe, Alan Ritter etc.):
probable response != good response这里的方法是用下列项目来描述good response:
- Reasonable: p(response|input) is high according to seq2seq model
- Nonrepetitive: similarity between response and previous messages is low
- Easy to answer: p("i don't know"|response) is low<br>

score function = reasonable_score + Nonrepetitive_sorce + EasyToAnswer_score<br>
Learm from rewards instend of from examples:
- Encode input into a vector
- Have the system generate a response(instead of passing in a human-generaetd or a response that a human said and try to increase this probability according to the model) and then give it a reward signal, which tells it, die it do a good job with the response or not, which is that scorign function mentioned earlier.
- Receive reward R(response): Train system to maximize reward

#### Discussion: simple RNN
- Decent results with single matrix TreeRNN
- Single weight matrix TreeRNN could capture some phenomena but note adequate for more complex, higher order composition and parsing long sentences
- There is no real interaction between the input word
- The composition function is the same for all syntactic categories punctuation, etc.

### Version 2: Syntactically-untied RNN
- A symbolic Context-free Grammar(CFG) backbone is adequate for the basic syntactic structure
- We use the discrete syntactic categories of the children to choose the composition matrix, rather than just always using the same W matrix for any cases of composition
- A TreeRNN can do better with different compositioin matrix for different syntactic environments
- The result give us a better semantics<br>
使用CFG的原因：And essentially what this model did was sort of argue that there's a resonable separation that can be made between syntax and semantics in the following sense. That there are sort of basic structure of languages(指syntacitc structure，如介词加名词短语形成介词短语). That kind of syntactic structure can be pretty well captured by actually a symbolic grammar. So we assumed in this model that we did have a symbolic context free grammar backbone that was adequate for basic syntactic structure<br>
基本思路就是，根据words对应的categories选择对应的W，进行运算。并根由symbolic grammar指示，确定新形成的短语的Category。<br>

#### Compositional Vector Grammars
Problem: Every candidate score in beam search needs a matrix-vector product.<br>
Solution: Compute score noly for a subset of trees coming from a simple, faster model(PCFG)
- Prunes very unlikely candidates for speed
- Provides coarse syntactic categories of the children for each beam candidate<br>
Compositional vector Grammars = PCFG + TreePNN